# Validación en entorno industrial

## Objetivo del estudio

1. **Verificar la calidad metrológica** de las medidas morfométricas obtenidas por visión artificial (AV) frente a medidas reales (REAL).
2. **Ajustar un modelo alométrico** para inferir peso a partir de longitud y anchura reales (modelo base), y aplicar la fórmula con dimensiones AV.
3. **Cuantificar el error del peso inferido** como variable intermedia para clasificación por tamaños.
4. **Analizar repetibilidad temporal** comparando resultados entre los 5 experimentos (días).
5. Proponer **criterios razonables de aceptación** en producción acuícola, orientados a clasificación por tamaños y a justificación de **TRL** en memoria CDTI.


````{admonition} Resumen 
:class: tip

Este informe presenta la validación en entorno productivo real del sistema FLATCLASS, el prototipo automático de clasificación de alevines de lenguado (*Solea senegalensis*) basado en visión artificial. La validación se realizó en las instalaciones de Satistela S.A. ([Grupo Sea8](https://https://seaeight.eu/contacto/)) en Póvoa de Varzim (Portugal), mediante un protocolo multi-experimento diseñado para cuantificar: 1) la precisión en la clasificación por intervalos de talla (pequeño, mediano y grande), y 2) la exactitud en la estimación individual de biomasa de cada individuo detectado. El estudio se apoya en 1.250 observaciones (5 experimentos, 250 peces) y evalúa: exactitud, precisión, estabilidad inter-experimento, acuerdo metrológico (Bland–Altman), heterocedasticidad y un análisis Monte Carlo del riesgo de error de clasificación inducido por la incertidumbre dimensional.

**Entregable**: E6.1  
**Versión**: 1.0  
**Autor**: Javier Álvarez Osuna  
**Email**: javier.osuna@fishfarmfeeder.com  
**ORCID**: [0000-0001-7063-1279](https://orcid.org/0000-0001-7063-1279)  
**Licencia**: CC-BY-4.0  
**Código proyecto**: IG408M.2025.000.000072

```{figure} .././assets/FLATCLASS_logo_publicidad.png
:width: 100%
:align: center
```
````

## Objetivo

El **objetivo general** planteado en la validación industrial de FLATCLASS reside en comprobar hasta qué punto el sistema de visión artificial desarrollado en el PT-1 es capaz de clasificar de forma fiable los juveniles de lenguado en las tres diferentes tallas (pequeño, normal, grande), teniendo en cuenta la incertidumbre inherente a las mediciones realizadas en condiciones reales de operación.

Para ello, se han planteado los siguientes **objetivos específicos** orientados a analizar, de manera estructurada y cuantitativa, los distintos factores que condicionan la fiabilidad del proceso de clasificación. Estos objetivos permiten descomponer el problema global en aspectos medibles relacionados con la calidad metrológica de las estimaciones dimensionales, su estabilidad temporal, el grado de acuerdo con las medidas de referencia y el impacto práctico de los errores de medición sobre la decisión final de asignación de talla.

 1. Caracterizar estadísticamente la exactitud y precisión de las mediciones dimensionales (longitud y anchura) obtenidas mediante el sistema de visión artificial, comparándolas con las medidas de referencia realizadas en laboratorio, con el fin de cuantificar el error medio, la dispersión y la presencia de posibles sesgos sistemáticos.
 2.	Evaluar el grado de acuerdo metrológico entre las medidas estimadas y las reales, utilizando análisis de Bland–Altman y métricas complementarias, para interpretar la incertidumbre del sistema directamente en unidades físicas relevantes para la clasificación por talla.
 3.	Analizar la estabilidad temporal del sistema de medición y clasificación, comparando el comportamiento estadístico de los errores a lo largo de varios experimentos consecutivos realizados en días distintos, con el objetivo de detectar posibles derivas o variaciones operativas.
 4.	Examinar la relación entre el tamaño del individuo y la magnitud del error de medida, identificando posibles efectos de heterocedasticidad y dependencias con la talla que puedan influir en la fiabilidad de la clasificación en distintos rangos de tamaño.
 5.	Evaluar el desempeño del peso inferido como variable secundaria, analizando su precisión y variabilidad, y determinar su idoneidad para la estimación de biomasa agregada por clase de talla, en lugar de su uso como criterio primario de clasificación individual.
 6.	Cuantificar el impacto práctico de la incertidumbre dimensional sobre la decisión de clasificación por talla, mediante simulaciones Monte Carlo que permitan estimar la probabilidad de cambio de clase inducida por los errores de medición.
 7.	Identificar los rangos de talla más sensibles a la inestabilidad de clasificación, especialmente en torno a los umbrales entre clases, con el fin de caracterizar las zonas de mayor riesgo de asignación incorrecta.
 8.	Proponer criterios estadísticos para el diseño y validación de los umbrales de clasificación por talla, basados en percentiles del error dimensional, que permitan reducir el riesgo de inestabilidad de clasificación en aplicaciones operativas.
 9.	Proporcionar una base metodológica reproducible para la validación de sistemas de clasificación por visión artificial, integrando análisis metrológico, estadístico y de toma de decisiones, aplicable a otros contextos industriales y especies acuícolas.

In [ ]:
# --- Imports y configuración ---
import os
import math
import glob
import shutil

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 140)


In [ ]:
# --- Carga del dataset ---
path = "Datos_experimentos_lenguado.xlsx"
df = pd.read_excel(path)

df.shape, df.columns.tolist()


In [ ]:
# --- Construcción de la variable Experimento ---
# Asunción: 5 experimentos en bloques consecutivos de 250 registros
n_per = 250
df = df.copy()
df["Experimento"] = (np.arange(len(df)) // n_per) + 1

df["Experimento"].value_counts().sort_index()


## Variables disponibles

El dataset contiene:

- `Peso (g)` : peso real (variable objetivo para validación)
- `Longitud_REAL (mm)`, `Anchura_REAL (mm)` : medidas reales (referencia)
- `Longitud_AV (mm)`, `Anchura_AV (mm)` : medidas obtenidas por visión artificial (AV)
- `Error_longitud_% (AV-REAL)`, `Error_anchura_% (AV-REAL)` : errores relativos metrológicos ya calculados
- `Longitud_px`, `Anchura_px` : medidas en píxeles (no utilizadas aquí para el modelo; el modelo opera en mm)
- `Experimento` : identificador construido (E1..E5)

### Enfoque
1. Ajuste del modelo con medidas **REAL** (para estimación robusta de parámetros).
2. Inferencia operativa usando dimensiones **AV**.
3. Evaluación del error relativo del peso inferido frente al peso real.


In [ ]:
# --- Preparación: comprobaciones y columnas logarítmicas ---
COL_W = "Peso (g)"
L_real = "Longitud_REAL (mm)"
A_real = "Anchura_REAL (mm)"
L_av = "Longitud_AV (mm)"
A_av = "Anchura_AV (mm)"

# Verificar que todas las filas son válidas para log (positivas y no nulas)
mask = (
    df[[COL_W, L_real, A_real, L_av, A_av]].notna().all(axis=1) &
    (df[[COL_W, L_real, A_real, L_av, A_av]] > 0).all(axis=1)
)
assert mask.all(), "Hay filas no válidas para log; revisa el dataset."

# Logs (para modelo alométrico)
df["_log_w"]   = np.log(df[COL_W])
df["_log_Lr"]  = np.log(df[L_real])
df["_log_Ar"]  = np.log(df[A_real])
df["_log_Lav"] = np.log(df[L_av])
df["_log_Aav"] = np.log(df[A_av])

df[[COL_W, L_real, A_real, L_av, A_av, "Experimento"]].head()


## Modelo alométrico (log–log) y corrección de retransformación

Se ajusta un modelo lineal en el espacio logarítmico:

\[
\log(P) = b_0 + b_1 \log(L) + b_2 \log(A) + \varepsilon
\]

donde:

- \(P\) es el peso (g),
- \(L\) longitud (mm),
- \(A\) anchura (mm).

La predicción en espacio real se obtiene como:

\[
\hat{P} = \exp(\hat{y})
\]

y, para reducir el sesgo por retransformación, se aplica una corrección multiplicativa tipo **smearing**:

\[
\hat{P}_{corr} = \exp(\hat{y}) \cdot S, \quad S = \frac{1}{n}\sum_i \exp(\hat{\varepsilon}_i)
\]

Este enfoque es especialmente útil en producción porque:
- ofrece **interpretabilidad** y **estabilidad**,
- es **eficiente computacionalmente** (Edge),
- facilita auditoría y recalibraciones controladas si fuese necesario.


In [ ]:
# --- Ajuste del modelo con dimensiones REAL (global) ---
Xr = sm.add_constant(df[["_log_Lr", "_log_Ar"]])
y = df["_log_w"]

model = sm.OLS(y, Xr).fit()
print(model.summary())

b0 = float(model.params["const"])
b1 = float(model.params["_log_Lr"])
b2 = float(model.params["_log_Ar"])

smearing = float(np.mean(np.exp(model.resid)))

k = float(np.exp(b0))
k_corr = k * smearing

print("\nParámetros:")
print(f"b0={b0:.6f}, b1={b1:.6f}, b2={b2:.6f}")
print(f"Smearing S={smearing:.6f}")
print("\nFórmula operativa (smearing incluido):")
print(f"Peso(g) = {k_corr:.12f} * L^{b1:.6f} * A^{b2:.6f}   (L,A en mm)")


In [ ]:
# --- Inferencia de peso usando dimensiones REAL y AV ---
def pred_weight_from_logs(logL, logA):
    return np.exp(b0 + b1 * logL + b2 * logA)

# Predicción (naive) y corregida (smearing)
df["Peso_inferido_REAL"] = pred_weight_from_logs(df["_log_Lr"], df["_log_Ar"])
df["Peso_inferido_REAL_smear"] = df["Peso_inferido_REAL"] * smearing

df["Peso_inferido_AV"] = pred_weight_from_logs(df["_log_Lav"], df["_log_Aav"])
df["Peso_inferido_AV_smear"] = df["Peso_inferido_AV"] * smearing

# Error relativo (%)
df["Error_rel_peso_REAL_pct"] = (np.abs(df["Peso_inferido_REAL_smear"] - df[COL_W]) / df[COL_W]) * 100
df["Error_rel_peso_AV_pct"]   = (np.abs(df["Peso_inferido_AV_smear"] - df[COL_W]) / df[COL_W]) * 100

df[[COL_W, "Peso_inferido_AV_smear", "Error_rel_peso_AV_pct"]].head()


## Tablas de resultados (estadísticos y coberturas)

Se generan tablas para:
- estadísticos de peso real,
- error metrológico de longitud/anchura (AV-REAL),
- error del peso inferido con AV,
- coberturas por umbrales (≤5%, ≤10%, ...),
- resultados por experimento (repetibilidad inter-día),
- resultados por rangos de peso (robustez por tamaño).


In [ ]:
def desc(s: pd.Series) -> pd.Series:
    return pd.Series({
        "N": int(s.count()),
        "Media": float(s.mean()),
        "Mediana": float(s.median()),
        "Desv.Típ.": float(s.std(ddof=1)),
        "P5": float(s.quantile(0.05)),
        "P95": float(s.quantile(0.95)),
        "Mín": float(s.min()),
        "Máx": float(s.max()),
    })

tab_peso = desc(df[COL_W]).to_frame("Peso (g)")
tab_errL = desc(df["Error_longitud_% (AV-REAL)"]).to_frame("Error_longitud_% (AV-REAL)")
tab_errA = desc(df["Error_anchura_% (AV-REAL)"]).to_frame("Error_anchura_% (AV-REAL)")
tab_errW_AV = desc(df["Error_rel_peso_AV_pct"]).to_frame("Error_rel_peso_AV_pct")

tab_peso.round(3), tab_errW_AV.round(3)


In [ ]:
# Cobertura acumulada por umbrales de error relativo (AV)
thresholds = [5, 10, 15, 20]
tab_cov = pd.DataFrame([{
    "Umbral (%)": t,
    "% individuos (<=umbral)": float((df["Error_rel_peso_AV_pct"] <= t).mean() * 100),
    "N (<=umbral)": int((df["Error_rel_peso_AV_pct"] <= t).sum()),
} for t in thresholds])

tab_cov.round(2)


In [ ]:
# Resumen por experimento (día)
per = df.groupby("Experimento")

tab_per_metrology = per.agg({
    "Error_longitud_% (AV-REAL)": ["mean", "median", lambda s: s.quantile(0.95)],
    "Error_anchura_% (AV-REAL)": ["mean", "median", lambda s: s.quantile(0.95)],
})
tab_per_metrology.columns = ["L_mean", "L_med", "L_P95", "A_mean", "A_med", "A_P95"]

tab_per_weight = per["Error_rel_peso_AV_pct"].agg(["count", "mean", "median", lambda s: s.quantile(0.95), "max"])
tab_per_weight.columns = ["N", "Media_%", "Mediana_%", "P95_%", "Max_%"]

tab_per_metrology.round(4), tab_per_weight.round(2)


In [ ]:
# Error por rangos de peso real (robustez por tamaño)
bins = [-np.inf, 2, 5, 10, np.inf]
labels = ["<2 g", "2–5 g", "5–10 g", ">=10 g"]

df["Rango_peso"] = pd.cut(df[COL_W], bins=bins, labels=labels)

grp = df.groupby("Rango_peso", observed=True)["Error_rel_peso_AV_pct"]
tab_bins = pd.DataFrame({
    "N": grp.count().astype(int),
    "Media_%": grp.mean(),
    "Mediana_%": grp.median(),
    "P95_%": grp.quantile(0.95),
    "Max_%": grp.max(),
}).round(2)

tab_bins


## Figuras (generación reproducible)

Se exportan las figuras a una carpeta local (`jbook_assets_exp`) para poder referenciarlas desde Jupyter-Book.


In [ ]:
# --- Generación de figuras ---
out_dir = "jbook_assets_exp"
os.makedirs(out_dir, exist_ok=True)

# Figura 4-1: distribución errores dimensionales
plt.figure()
plt.hist(df["Error_longitud_% (AV-REAL)"], bins=40, alpha=0.7, label="Error longitud (%)")
plt.hist(df["Error_anchura_% (AV-REAL)"], bins=40, alpha=0.7, label="Error anchura (%)")
plt.xlabel("Error relativo (%)")
plt.ylabel("Frecuencia")
plt.title("Distribución del error relativo en medidas AV vs REAL (5 experimentos)")
plt.legend()
fig1 = os.path.join(out_dir, "fig_4_1_error_dimensiones_5exp.png")
plt.tight_layout(); plt.savefig(fig1, dpi=200); plt.close()

# Figura 4-2: histograma error de peso AV
plt.figure()
plt.hist(df["Error_rel_peso_AV_pct"], bins=50)
plt.xlabel("Error relativo del peso inferido (%)")
plt.ylabel("Frecuencia")
plt.title("Histograma del error relativo del peso inferido (dimensiones AV)")
fig2 = os.path.join(out_dir, "fig_4_2_hist_error_peso_5exp.png")
plt.tight_layout(); plt.savefig(fig2, dpi=200); plt.close()

# Figura 4-3: boxplot error por experimento
plt.figure()
data_box = [df.loc[df["Experimento"] == i, "Error_rel_peso_AV_pct"].values for i in range(1, 6)]
plt.boxplot(data_box, labels=[f"E{i}" for i in range(1, 6)], showfliers=True)
plt.xlabel("Experimento (día)")
plt.ylabel("Error relativo del peso inferido (%)")
plt.title("Variabilidad inter-experimento del error de peso (AV)")
fig3 = os.path.join(out_dir, "fig_4_3_boxplot_error_peso_por_experimento.png")
plt.tight_layout(); plt.savefig(fig3, dpi=200); plt.close()

# Figura 4-4: boxplot por rango de peso
plt.figure()
data_box = [df.loc[df["Rango_peso"] == lbl, "Error_rel_peso_AV_pct"].values for lbl in ["<2 g", "2–5 g", "5–10 g", ">=10 g"]]
plt.boxplot(data_box, labels=["<2 g", "2–5 g", "5–10 g", ">=10 g"], showfliers=True)
plt.xlabel("Rango de peso real")
plt.ylabel("Error relativo del peso inferido (%)")
plt.title("Error de peso inferido (AV) por rangos de peso real")
fig4 = os.path.join(out_dir, "fig_4_4_boxplot_error_por_rango_peso.png")
plt.tight_layout(); plt.savefig(fig4, dpi=200); plt.close()

# Figura 4-5: scatter peso inferido vs real por experimento
plt.figure()
for i in range(1, 6):
    sub = df[df["Experimento"] == i]
    plt.scatter(sub[COL_W], sub["Peso_inferido_AV_smear"], s=10, label=f"E{i}", alpha=0.7)
mx = float(np.nanmax([df[COL_W].max(), df["Peso_inferido_AV_smear"].max()]))
plt.plot([0, mx], [0, mx])
plt.xlabel("Peso real (g)")
plt.ylabel("Peso inferido (g)")
plt.title("Peso inferido vs real por experimento (AV)")
plt.legend()
fig5 = os.path.join(out_dir, "fig_4_5_scatter_peso_real_vs_inferido_por_experimento.png")
plt.tight_layout(); plt.savefig(fig5, dpi=200); plt.close()

# Figura 4-6: evolución media y P95 por experimento
plt.figure()
x = np.arange(1, 6)
plt.plot(x, tab_per_weight["Media_%"].values, marker="o", label="Media (%)")
plt.plot(x, tab_per_weight["P95_%"].values, marker="o", label="P95 (%)")
plt.xlabel("Experimento (día)")
plt.ylabel("Error relativo (%)")
plt.title("Evolución de error de peso (media y P95) por experimento")
plt.legend()
fig6 = os.path.join(out_dir, "fig_4_6_media_p95_por_experimento.png")
plt.tight_layout(); plt.savefig(fig6, dpi=200); plt.close()

[fig1, fig2, fig3, fig4, fig5, fig6]


## Discusión técnico-operativa (orientación CDTI)

### Lecturas principales (resumen)

- **Metrología (AV vs REAL):** los errores relativos de longitud y anchura permanecen contenidos y consistentes a lo largo de 5 días, lo que aporta evidencia de **repetibilidad del pipeline de visión** con configuración constante.
- **Error de peso inferido (AV):** el error relativo presenta estabilidad central compatible con clasificación por tamaños. La dispersión en el extremo bajo (<2 g) es esperable por naturaleza geométrica y biológica.
- **Repetibilidad inter-día:** la comparación por experimento (media/mediana/P95) permite argumentar **robustez temporal sin recalibración**, aspecto especialmente relevante para justificar TRL en memoria CDTI.
- **Criterios de aceptación recomendados para clasificación:** mediana ≤10%, cobertura ≤10% ≥55–65%, P95 ≤25%, apoyado por QA para casos atípicos.

### TRL (síntesis)
Con la evidencia multi-día y el modelo único sin recalibración, la tecnología se sitúa en **TRL 6–7**, dependiendo del grado de equivalencia del banco de ensayo con la línea real (cadencia, manipulación, iluminación, etc.).

### Recomendación para reforzar TRL 7→8
- Añadir trazas operativas del entorno (parámetros de captura, control QA, incidencias)
- Pilotos adicionales bajo variabilidad controlada (lote, densidad, iluminación)
- Integración final con criterio de clasificación (umbrales por categoría) y medición de tasa de recaptura/“errores”


In [ ]:
# (Opcional) Exportar un Excel enriquecido con columnas calculadas para auditoría/QA
out_xlsx = "Datos_experimentos_lenguado_enriquecido.xlsx"
df.to_excel(out_xlsx, index=False)
out_xlsx
